<a href="https://colab.research.google.com/github/NadjaKljestanovic/PPPO/blob/main/DetekcijaObjekata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install gdown

In [7]:
import gdown

In [10]:
import os
import sys
import random
import math
import warnings
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd
import glob
import numpy as np
import pandas as pd
from skimage.io import imread
from matplotlib.cm import get_cmap
from skimage.segmentation import mark_boundaries
from skimage.util import montage
from skimage.morphology import binary_opening, disk, label
import gc; gc.enable()

In [11]:
!git clone https://github.com/maxw1489/Mask_RCNN.git
os.chdir('Mask_RCNN')

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 31 (delta 6), reused 20 (delta 0), pack-reused 0
Receiving objects: 100% (31/31), 10.81 MiB | 22.27 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [13]:
sys.path.append(os.path.join('Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [14]:
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
!ls -lh mask_rcnn_coco.h5

-rw-r--r-- 1 root root 246M Dec  6  2021 mask_rcnn_coco.h5


In [15]:
COCO_WEIGHTS_PATH = "mask_rcnn_coco.h5"

In [16]:
data_dir = '/content/drive/MyDrive/data'

In [17]:
main_dir = '/content/drive/MyDrive'

In [22]:
train_dir = os.path.join(data_dir, 'training_images')

In [23]:
test_dir = os.path.join(data_dir, 'testing_images')

In [27]:
import os

train_images = os.listdir(train_dir)
num_train_images = len(train_images)

test_images = os.listdir(test_dir)
num_test_images = len(test_images)

In [28]:
print("Količina slika za trening:", num_train_images)

Količina slika za trening: 1001


In [29]:
print("Korličina slika za test:", num_test_images)

Korličina slika za test: 175


In [31]:
import pandas as pd
import os

In [32]:
csv_test = os.path.join(data_dir, 'sample_submission.csv')

In [33]:
csv_train = os.path.join(data_dir, 'train_solution_bounding_boxes (1).csv')

In [35]:
dataTestCsv = pd.read_csv(csv_test)
dataTrainCsv = pd.read_csv(csv_train)

In [36]:
print("\nCsv test fajl:")
print(dataTestCsv)

print("\nCsv trening fajl:")
print(dataTrainCsv)


Csv test fajl:
              image                           bounds
0   vid_5_26560.jpg  0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0
1   vid_5_26580.jpg  0.0 0.0 1.0 1.0 0.0 0.0 1.0 1.0
2   vid_5_26600.jpg                  0.0 0.0 1.0 1.0
3   vid_5_26620.jpg                  0.0 0.0 1.0 1.0
4   vid_5_26640.jpg                  0.0 0.0 1.0 1.0
..              ...                              ...
78  vid_5_31560.jpg                  0.0 0.0 1.0 1.0
79  vid_5_31600.jpg                  0.0 0.0 1.0 1.0
80  vid_5_31620.jpg                  0.0 0.0 1.0 1.0
81  vid_5_31700.jpg                  0.0 0.0 1.0 1.0
82  vid_5_31720.jpg                  0.0 0.0 1.0 1.0

[83 rows x 2 columns]

Csv trening fajl:
               image        xmin        ymin        xmax        ymax
0     vid_4_1000.jpg  281.259045  187.035071  327.727931  223.225547
1    vid_4_10000.jpg   15.163531  187.035071  120.329957  236.430180
2    vid_4_10040.jpg  239.192475  176.764801  361.968162  236.430180
3    vid_4_10020.jpg  496.4833

In [37]:
import pandas as pd

In [42]:
csv_train = os.path.join(data_dir, 'train_solution_bounding_boxes (1).csv')
df_train = pd.read_csv(csv_train)

In [43]:
class_counts1 = df_train['xmin'].notnull().value_counts()

In [44]:
print("Raspodela klasa - xmin:")
print(class_counts1)

Raspodela klasa:
True    559
Name: xmin, dtype: int64


In [45]:
class_counts2 = df_train['xmax'].notnull().value_counts()

In [46]:
print("Raspodela klasa - xmax:")
print(class_counts2)

Raspodela klasa - xmax:
True    559
Name: xmax, dtype: int64
